In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('../data/data.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


In [5]:
df = df.dropna(axis=0, subset=('question1','question2','is_duplicate' ))

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


In [7]:
#lower casing
def lower_case(text):
    return text.lower()


#remove numbers
def remove_numbers(text):
    output = re.sub(r'\d+', '', text)
    return output

# remove punctuation
import string
def remove_punctuation(text):
    text_p = "".join([char for char in text if char not in string.punctuation])
    return text_p

#tokenize text

import nltk
nltk.download('punkt')
from nltk import word_tokenize

def tokenize(text):
    words = word_tokenize(text)
    return words

#remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stopwords(text):
    filtered_words = [word for word in text if word not in stop_words]
    return filtered_words

#stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    stemmed = [stemmer.stem(word) for word in text]
    return stemmed

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
class Preprocessor():
    def __init__(self):
        pass
    
    def preprocess(self,df_column, steps):
        self.df_column = df_column
        self.steps = steps
    
        if 'lower_case' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lower_case(x))
        
        
        if 'remove_numbers' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_numbers(x))
        
        if 'remove_punctuation' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_punctuation(x))           
        
        if 'tokenize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: tokenize(x))
        
        if 'stopwords' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_stopwords(x))
        
        if 'stemming' in self.steps:
            self.df_column = self.df_column.apply(lambda x: stem_words(x))
            
        return self.df_column    

In [9]:
steps = ['lower_case','remove_numbers','remove_punctuation',
        'tokenize','stopwords','stemming']
processor = Preprocessor()
df['question1'] = processor.preprocess(df['question1'] ,steps)

In [10]:
df['question2'] = processor.preprocess(df['question2'] ,steps)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


In [12]:
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 19.4+ MB


In [13]:
df.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0
2,2,360472,364011,490273.0,"[caus, stool, color, chang, yellow]","[caus, stool, come, littl, ball]",0.0
3,3,150662,155721,7256.0,"[one, mbb]",[mbb],1.0
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourn, airport]","[would, second, airport, sydney, australia, ne...",0.0


In [14]:
def calc_tf(alist):
    
    tf_list = [alist.count(term)/len(alist) for term in alist]
    return tf_list


In [15]:
df['q2_tf'] = df['question2'].apply(lambda x: calc_tf(x))

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
 7   q2_tf         363177 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 22.2+ MB


In [17]:
def get_Doc_ids(vocabulary_list,dataframe,tokens_col_name, id_col_name):
    Doc_ids = {}      #initialise empty document frequency dictionary
    
    #for each word in vocabulary, create key:value pair, with an empty list as value
    for word in vocabulary_list:
        Doc_ids[word] = [] 
        
    #for each word in DF, add the id of question2 where it appears. Create a set to avoid duplicate ids
    for i in range(len(dataframe)):
        for word in dataframe.loc[i,tokens_col_name]:
            Doc_ids[word].append(dataframe[id_col_name][i])
    for word in Doc_ids.keys():
        Doc_ids[word] = list(set(Doc_ids[word]))
        #DF[word]  =len(DF[word])
        
        
    
    return Doc_ids


In [18]:
q2vocab = df['question2'].tolist()
q2vocab = [item for sublist in q2vocab for item in sublist]
q2vocab = list(set(q2vocab))
Doc_ids = get_Doc_ids(q2vocab,df,'question2','qid2')
Doc_ids

{'stayuncl': [9784.0],
 'articlesscripturespoemspros': [364580.0],
 'perfectli': [61315.0,
  99076.0,
  471430.0,
  408199.0,
  202636.0,
  177807.0,
  358162.0,
  460946.0,
  214165.0,
  390167.0,
  63127.0,
  530329.0,
  56730.0,
  33699.0,
  456745.0,
  10666.0,
  310959.0,
  56751.0,
  519477.0,
  470198.0,
  292281.0,
  495037.0,
  167998.0,
  126274.0,
  507458.0,
  230980.0,
  413262.0,
  493264.0,
  506707.0,
  269908.0,
  306647.0,
  436823.0,
  159319.0,
  155100.0,
  134493.0,
  401502.0,
  51424.0,
  188898.0,
  222182.0,
  470248.0,
  366956.0,
  147053.0,
  537836.0,
  463473.0,
  297587.0,
  268916.0,
  230773.0,
  219384.0,
  24572.0,
  31102.0],
 'luwin': [462813.0],
 'neu': [232682.0],
 'alka': [407091.0, 88551.0],
 'sycophant': [297881.0],
 'koramangala': [191861.0, 284862.0, 389343.0],
 'crimedramamysterythril': [216600.0],
 'irrespect': [150496.0,
  29911.0,
  94434.0,
  352198.0,
  78093.0,
  513230.0,
  441364.0,
  131318.0,
  325463.0,
  128991.0,
  407996.0,
  

In [19]:
DF = {}
for word in Doc_ids.keys():
    DF[word] = len(Doc_ids[word])
DF    

{'stayuncl': 1,
 'articlesscripturespoemspros': 1,
 'perfectli': 50,
 'luwin': 1,
 'neu': 1,
 'alka': 2,
 'sycophant': 1,
 'koramangala': 3,
 'crimedramamysterythril': 1,
 'irrespect': 13,
 'thear': 1,
 'oauth': 9,
 'ccm': 1,
 'alias': 3,
 'goliath': 3,
 'week': 806,
 'seasick': 1,
 'ballsiest': 1,
 'oceanographi': 5,
 'wrangel': 1,
 'lessfavour': 1,
 'ecocrit': 1,
 'bebtech': 1,
 'floppi': 8,
 'bookread': 1,
 'bearof': 1,
 'heteronym': 1,
 'zanerblos': 1,
 'hint': 20,
 'xzibit': 1,
 'redio': 1,
 'yv': 1,
 'millisecond': 1,
 'datingcom': 1,
 'اشعر': 1,
 'jailbreak': 84,
 'maruthi': 1,
 'poi': 1,
 'etch': 1,
 'armi': 435,
 'unbounc': 1,
 'hypothesis': 1,
 'posteriori': 2,
 'balaji': 35,
 'themesmean': 1,
 'technicalengin': 1,
 'shahdah': 1,
 'applecom': 1,
 'yearback': 1,
 'betcom': 2,
 'nip': 1,
 'pipo': 1,
 'congress': 108,
 'parentsinlaw': 1,
 'tvprogram': 1,
 'kendta': 1,
 'icaew': 1,
 'wallapop': 1,
 'own': 139,
 'michelsonmorley': 2,
 'pgdie': 4,
 'outlier': 3,
 '有緣無份': 1,
 'watch

In [20]:
# for doc_ids in Doc_ids.values():
#     tf_vals = []
#     for doc_id in doc_ids:
#         tf_vals.append()
        

In [21]:
# IF = {}
# for word in q2vocab:
#     IF[word] = {}
#     IF[word].update(Doc_ids[word])
   

In [22]:
list1 = list(df['question2'])
list2 = list(df['q2_tf'])
list_dicts = []
for i in range(len(df)):
    adict = dict(zip(list1[i],list2[i]))
    list_dicts.append(adict)
    
    

In [23]:
doc_word_tf = dict(zip(df['qid2'], list_dicts))

### dataframe
col1: doc_id, col_2: list of words, col_3: list of tf



### dictionaries

Doc_ids 
keys: word, values: list of doc ids

doc_word_tf
keys: doc_id, values: dictionary - keys: word, values: tf

DF
keys: word, values: DF

Required - Inverted file
keys: word, values: DF, dictionary - keys: doc_id, values: tf



In [24]:
df.head(2)

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_tf
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0,"[0.2, 0.2, 0.2, 0.2, 0.2]"
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0,"[0.5, 0.5]"


In [25]:
df['q2_len'] = df['question2'].apply(lambda x: len(x) )
df.head()

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_tf,q2_len
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0,"[0.2, 0.2, 0.2, 0.2, 0.2]",5
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0,"[0.5, 0.5]",2
2,2,360472,364011,490273.0,"[caus, stool, color, chang, yellow]","[caus, stool, come, littl, ball]",0.0,"[0.2, 0.2, 0.2, 0.2, 0.2]",5
3,3,150662,155721,7256.0,"[one, mbb]",[mbb],1.0,[1.0],1
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourn, airport]","[would, second, airport, sydney, australia, ne...",0.0,"[0.08333333333333333, 0.08333333333333333, 0.0...",12


In [26]:
doc_words = dict(zip(df['qid2'],df['question2']))
doc_words[213222.0].count('level')

1

In [27]:
tf_lists = {}
for word in q2vocab:
    word_tf = []
    for doc in Doc_ids[word]:
        tf = doc_words[doc].count(word)/len(doc_words[doc])
        word_tf.append(tf)
    tf_lists[word] = word_tf    

In [28]:
df[df['qid2']== 17137.0]

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_tf,q2_len
39784,39786,10333,13013,17137.0,"[counti, edgwar, lifestyl, compar, london, bor...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
57359,57361,374626,505537,17137.0,"[counti, edgwar, lifestyl, compar, london, bor...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
64174,64176,181463,4854,17137.0,"[counti, edgwar, lifestyl, compar, london, bor...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
67038,67040,156500,13014,17137.0,"[‘, edgwar, ’, lifestyl, compar, london, borou...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
138914,138920,352243,4853,17137.0,"[district, edgwar, lifestyl, compar, london, b...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
184961,184968,129787,208419,17137.0,"[edgwar, lifestyl, compar, london, borough, su...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
251639,251649,288243,237821,17137.0,"[counti, edgwar, lifestyl, compar, london, bor...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
288663,288673,236635,347605,17137.0,"[edgwar, like, lifestyl, compar, oxford]","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4
313560,313572,146088,230842,17137.0,"[edgwar, like, lifestyl, compar, london, borou...","[edgwar, compar, neighbourhood, london]",0.0,"[0.25, 0.25, 0.25, 0.25]",4


In [29]:
zipped = list(zip(Doc_ids.values(), tf_lists.values()))


In [55]:
double_zipped = []
for alist in zipped:
    zippit = list(zip(alist[0],alist[1]))
    double_zipped.append(zippit)
    
    
    

In [56]:
double_zipped[0]

[(9784.0, 0.125)]

In [57]:
inverted_file = {}
for i in range(len(q2vocab)):
    inverted_file[q2vocab[i]] = double_zipped[i]
    

In [58]:
N = df['qid2'].nunique()
N

273121

In [59]:
IDF = {}
for i in range(len(q2vocab)):
    IDF[q2vocab[i]] = N/DF[q2vocab[i]]


In [67]:
IDF[q2vocab[0]]

273121.0

In [45]:
# for i in range(len(q2vocab)):
#     inverted_file[q2vocab[i]].append(IDF[q2vocab[i]])

In [99]:
for tup in inverted_file[q2vocab[0]]:
    tf_idf = tup[1]*IDF[q2vocab[0]]
    tup += (tf_idf,)
    inverted_file[q2vocab[0]].update(tup)

    
inverted_file[q2vocab[0]]   

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [83]:

inverted_file[q2vocab[0]]

[(9784.0, 0.125)]

In [75]:
for i in range(len(inverted_file)):
    for tup in inverted_file[q2vocab[i]]:
        tf_idf = tup[1]* IDF[q2vocab[i]]
        tup = tup + (tf_idf,)
        invert
    
    

In [76]:
inverted_file[q2vocab[1]]

[(364580.0, 0.25)]